In [ ]:
## GCS Bucket pull

import google.datalab.storage as storage
import pandas as pd
from io import BytesIO

mybucket = storage.Bucket('ceasars-test-dco-anova')
data_csv = mybucket.object('6469_Caesars_Dynamic_Elements_20181103_055047_2003198075.csv')

uri = data_csv.uri
%gcs read --object $uri --variable data

df = pd.read_csv(BytesIO(data),skiprows=81)
df.head()

In [1]:
## BQ pull with SQL

# (Spin up Datalab VM on Cloud Shell with a command similar to the following:)
#datalab create babyweight --zone us-central1-a --no-create-repository


# Python (will run into a credential issue if not run on a Datalab VM spun up on project with BQ)
!pip install --upgrade google-cloud-bigquery

from google.cloud import bigquery

client = bigquery.Client()

sql = """
  SELECT
    plurality,
    COUNT(1) AS count,
    year
  FROM
    `bigquery-public-data.samples.natality`
  WHERE
    NOT IS_NAN(plurality) AND plurality > 1
  GROUP BY
    plurality, year
  ORDER BY
    count DESC
"""
df = client.query(sql).to_dataframe()
df.head()

pivot_table = df.pivot(index='year', columns='plurality', values='count')
pivot_table.plot(kind='bar', stacked=True, figsize=(15,7));

    100% |████████████████████████████████| 92kB 2.7MB/s ta 0:00:01
  Found existing installation: google-cloud-bigquery 1.8.1
    Uninstalling google-cloud-bigquery-1.8.1:
      Successfully uninstalled google-cloud-bigquery-1.8.1
You are using pip version 18.1, however version 19.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://developers.google.com/accounts/docs/application-default-credentials.